In [51]:
# Install dependencies
!pip install pathway bokeh geopy --quiet


In [53]:
# Imports
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
from time import sleep


In [54]:
#reading and processing
df = pd.read_csv('/content/dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')


In [55]:

df = df.sort_values(['Timestamp', 'SystemCodeNumber'])

In [59]:
# essential values
default_price = 10
alpha = 5
sensitivity = 0.5

weight_occupancy = 0.6
weight_queue = 0.3
weight_traffic = 0.4
weight_special_day = 1.0

vehicle_impact = {
    'car': 1,
    'bike': 0.5,
    'truck': 1.5
}

traffic_map = {
    'Low': 0,
    'Average': 1,
    'High': 2
}




In [60]:
#Model 1
def baseline_pricing(current_occupancy, max_capacity, previous_price=default_price):
    if max_capacity == 0:
        return default_price
    raw_price = previous_price + alpha * (current_occupancy / max_capacity)
    return max(5, min(20, round(raw_price, 2)))



In [61]:
#Model 2
def compute_demand(lot_data):
    if lot_data['Capacity']:
        occupancy_ratio = lot_data['Occupancy'] / lot_data['Capacity']
        occupancy_component = weight_occupancy * occupancy_ratio
    else:
        occupancy_component = 0

    queue_component = weight_queue * lot_data['QueueLength']
    traffic_level = str(lot_data['TrafficConditionNearby']).capitalize()
    traffic_val = traffic_map.get(traffic_level, 1)
    traffic_component = -weight_traffic * traffic_val
    special_day = weight_special_day * int(lot_data['IsSpecialDay'])
    veh_type = lot_data['VehicleType'].lower()
    vehicle_component = vehicle_impact.get(veh_type, 1)

    demand_total = (
        occupancy_component +
        queue_component +
        traffic_component +
        special_day +
        vehicle_component
    )

    return demand_total

def demand_based_price(demand_val):
    normalized = (demand_val - 1) / 9
    adjusted = default_price * (1 + sensitivity * normalized)
    return max(5, min(20, round(adjusted, 2)))


In [62]:
#Model 3
def get_nearby_lots(all_lots_df, this_lot, radius_km=0.5):
    my_lat = this_lot['Latitude']
    my_lon = this_lot['Longitude']
    nearby_lots = []

    for _, lot in all_lots_df.iterrows():
        if lot['SystemCodeNumber'] == this_lot['SystemCodeNumber']:
            continue
        try:
            dist = geodesic((my_lat, my_lon), (lot['Latitude'], lot['Longitude'])).km
            if dist <= radius_km:
                nearby_lots.append(lot)
        except:
            pass

    return pd.DataFrame(nearby_lots)



In [66]:
def competitive_adjustment(my_price, other_lot_prices):
    if len(other_lot_prices) == 0:
        return my_price
    avg_competitor_price = other_lot_prices['price'].mean()
    if my_price > avg_competitor_price:
        return max(5, my_price - 1)
    elif my_price < avg_competitor_price:
        return min(20, my_price + 1)
    return my_price

In [67]:
#graph plotting
sources = {}
plots = {}
lot_ids = df['SystemCodeNumber'].unique()

for lot_id in lot_ids:
    sources[lot_id] = ColumnDataSource(data={'x': [], 'self_price': [], 'comp_price': []})
    p = figure(title=f"Lot {lot_id}: Price vs Competitors", x_axis_type="datetime", width=600, height=300)
    p.line(x='x', y='self_price', color='blue', legend_label='Our Price', source=sources[lot_id])
    p.line(x='x', y='comp_price', color='red', legend_label='Competitor Avg Price', source=sources[lot_id])
    p.legend.location = 'top_left'
    plots[lot_id] = p

layout = column(list(plots.values()))
handle = show(layout, notebook_handle=True)


In [69]:
# real-time processing data on graph
price_memory = {}

for time_point in sorted(df['Timestamp'].unique()):
    snapshot = df[df['Timestamp'] == time_point].copy()

    for idx, row in snapshot.iterrows():
        lot_id = row['SystemCodeNumber']
        prev_price = price_memory.get(lot_id, default_price)

        demand = compute_demand(row)
        price = demand_based_price(demand)

        competitors = get_nearby_lots(snapshot, row)
        if not competitors.empty:
            competitors['price'] = competitors.apply(
                lambda r: demand_based_price(compute_demand(r)), axis=1
            )
            avg_comp_price = competitors['price'].mean()
            price = competitive_adjustment(price, competitors)
        else:
            avg_comp_price = price

        price_memory[lot_id] = price
        sources[lot_id].stream({
            'x': [time_point],
            'self_price': [price],
            'comp_price': [avg_comp_price]
        }, rollover=100)

    push_notebook(handle=handle)
